In [1]:
library(Seurat)
library(Matrix)
#library(DoubletDecon)
library(fields)
library(KernSmooth)
library(ROCR)
library(modes)
library(gplots)
library(ggplot2)
library(RColorBrewer)
library(pheatmap)
library(dplyr)
library(dat)
library(tidyverse)
library(VennDiagram)
library(plyr)

Loading required package: spam
Loading required package: dotCall64
Loading required package: grid
Spam version 2.2-2 (2019-03-07) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: 'spam'

The following object is masked from 'package:Matrix':

    det

The following objects are masked from 'package:base':

    backsolve, forwardsolve

Loading required package: maps
See https://github.com/NCAR/Fields for
 an extensive vignette, other supplements and source code 
KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009
Loading required package: gplots

Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':


In [58]:
train_test_validate_split=function(df,tr_pc=.7,tst_pc=.15,vl_pc=.15){

colnames(df)=gsub(x=colnames(df),pattern="X",replacement="")
colnames(df)=gsub(x=colnames(df),pattern="\\.",replacement="\\-")

spec = c(train = tr_pc, test = tst_pc, validate = vl_pc)


df_name=deparse(substitute(df))

g = sample(cut(
  seq(ncol(df)), 
  ncol(df)*cumsum(c(0,spec)),
  labels = names(spec)
))

res = list(df[,g=="train"],df[,g=="validate"],df[,g=="test"])
res
#assign(paste(df_name,"train",sep="."),df[,g=="train"])
#assign(paste(df_name,"train",sep="."),GetAssayData(NormalizeData(CreateSeuratObject(eval(parse(text=paste(df_name,"train",sep=".")))),method="CLR")))
#assign(paste(df_name,"validate",sep="."),df[,g=="validate"])
#assign(paste(df_name,"validate",sep="."),GetAssayData(NormalizeData(CreateSeuratObject(eval(parse(text=paste(df_name,"validate",sep=".")))),method="CLR")))
#assign(paste(df_name,"test",sep="."),df[,g=="test"])
#assign(paste(df_name,"test",sep="."),GetAssayData(NormalizeData(CreateSeuratObject(eval(parse(text=paste(df_name,"test",sep=".")))),method="CLR")))
}


In [3]:
setwd("/projects/ucar-lab/danaco/bncmrk-dblts/DF/input/CZI.PBMC/")
fl=list.files(pattern="preprocessed.seurat.for.DF.output")
#print("read fl")
for (i in seq(1,length(fl),2)){
 #   print(i)
    #name=strsplit(fl[i],split=".preprocessed")[[1]][1]
    name1=strsplit(fl[i],split=".preprocessed")[[1]][1]
    name2=strsplit(fl[i+1],split=".preprocessed")[[1]][1]

  #  print(name)
  #  writeMM(GetAssayData(readRDS(fl[i])),paste0(name,".mtx"))
    #print(strsplit(fl[i],split=".preprocessed")[[1]][1])
    #res=train_test_validate_split(data.frame(GetAssayData(readRDS(fl[i]))))
    #assign(paste0(name,".train"),res[1])
    #write.csv(eval(substitute(paste0(name,".train"))),paste0(name,".train.csv"))

    #assign(paste0(name,".validate"),res[2])
    #write.csv(eval(substitute(paste0(name,".validate"))),paste0(name,".validate.csv"))

    #assign(paste0(name,".test"),res[3])
    #write.csv(eval(substitute(paste0(name,".test"))),paste0(name,".test.csv"))

}

#czi.1.umis=readRDS("/projects/ucar-lab/danaco/bncmrk-dblts/DF/input/CZI.PBMC/CZI.PBMC.1.preprocessed.seurat.for.DF.output.Rds")

In [34]:
df=data.frame(CZI.PBMC.1.umis)
tr_pc=.7
tst_pc=.15
vl_pc=.15

colnames(df)=gsub(x=colnames(df),pattern="X",replacement="")
colnames(df)=gsub(x=colnames(df),pattern="\\.",replacement="\\-")

spec = c(train = tr_pc, test = tst_pc, validate = vl_pc)


df_name=deparse(substitute(df))

g = sample(cut(
  seq(ncol(df)), 
  ncol(df)*cumsum(c(0,spec)),
  labels = names(spec)
))


res = list(df[,g=="train"],df[,g=="validate"],df[,g=="test"])

res


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”

In [46]:
res=data.frame(train=df[,g=="train"],validate=df[,g=="validate"],test=df[,g=="test"])

In [48]:
res=list(df[,g=="train"],df[,g=="validate"],df[,g=="test"])

In [52]:
dim(res[[3]])

[1] 18489  4817

In [6]:
bebe=readRDS(fl[1])

In [17]:
head(Embeddings(bebe, reduction = "pca"))


,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,⋯,PC_41,PC_42,PC_43,PC_44,PC_45,PC_46,PC_47,PC_48,PC_49,PC_50
1-Sample_AAACCTGAGAATGTGT,4.119822,1.34686057,0.29988768,0.1900940,0.73526902,0.05997693,-0.07501228,-0.22889221,-0.005076993,0.23514377,,0.081807471,-0.29237997,-0.084293088,0.20410266,-0.585889048,-0.09733811,0.04190208,0.08798593,-0.335347725,0.110173964
1-Sample_AAACCTGAGACTGTAA,3.985693,1.21238499,0.59539024,0.5125112,0.48970012,0.74778744,-0.49468273,0.06685425,0.066616098,-0.24729326,,0.001732234,-0.03067066,-0.271026707,0.04181737,-0.122886088,0.19150909,0.12210428,-0.12079861,-0.007299907,-0.235800265
1-Sample_AAACCTGAGATGCCAG,5.827993,1.27451244,0.51927505,0.2211508,0.19279597,-0.38710322,-0.15243964,0.21455076,0.141232711,0.15958110,,-0.008915244,0.06356982,0.001560275,-0.01957060,-0.011014371,0.01491768,0.01866566,0.01021298,0.042511296,0.008619145
1-Sample_AAACCTGAGCAAATCA,4.711009,-0.01661911,0.03890151,0.5064376,0.39387938,-0.13462247,-0.12310370,-0.15587080,0.691680628,0.24697591,,-0.073637235,-0.03992108,-0.132664838,-0.19421738,-0.004581572,0.04495302,-0.00359695,-0.16547864,-0.143143297,0.055862003
1-Sample_AAACCTGAGCGATATA,5.268650,1.30169721,0.35254253,0.2272890,0.03917313,-0.46678424,0.16364348,-0.11100966,-0.115292847,0.21614940,,0.162251912,-0.08660761,0.094108427,-0.08485685,-0.028179306,-0.02096181,0.23524928,-0.02003987,-0.064171706,0.043478825
1-Sample_AAACCTGAGCGTGAAC,4.839211,0.91565781,1.04709363,0.2604204,0.53295024,0.47529427,0.13537013,0.28870554,0.076908472,0.06437109,,-0.998222481,0.74905360,-0.123063333,0.14498767,-0.109944277,0.14062472,-0.20474867,0.60147916,-0.228361973,-0.144896425


In [20]:
brcds_1=rownames(Embeddings(bebe, reduction = "umap"))

In [23]:
max(Embeddings(bebe, reduction = "umap"))

[1] 11.35771